## **利用ogr开源库进行矢量文件的读写**
**1. 投影转换**  
**2. 计算面要素的面积信息，添加面积字段/删除面积字段**  
**3. 获取面要素中心点，新建中心点图层**  
**4. 根据地理信息新建面要素**  


In [8]:
from osgeo import ogr
from osgeo import osr
import geopandas as gpd


In [9]:
path_kunming = 'data/Section-2/kunming_districts.shp'
path_kunming_utm = 'data/Section-4/kunming_districts_utm.shp'
path_kunming_centroid = 'data/Section-4/kunming_districts_wgs84_centroid.shp'
path_ynu_vector = 'data/Section-4/ynu_vec_create.shp'


### 1. Re-project the vector to utm projection.
EPSG是European Petroleum Survey Group (欧洲石油调查小组)的缩写,该组织发布了一个坐标参照系统的数据集,数据集对全球收录到的坐标参照系统进行了编码。  
wgs84: 4326; wgs84+utm: 326+投影带（如32622）


In [10]:
### Reprojection
data = gpd.read_file(path_kunming)
data = data.to_crs(epsg=32648)
data.to_file(path_kunming_utm)


In [11]:
ds = ogr.Open(path_kunming_utm, 1)  ## 0是只读，1是可写
print('Number of layers:', ds.GetLayerCount())       ## 图层个数
layer = ds.GetLayer(0)
print('Number of Features:', layer.GetFeatureCount())  ## 图层0中要素个数
fea = layer.GetFeature(0)
extent = layer.GetExtent()
print('vector extent: ', extent)
print('Number of Fields:', fea.GetFieldCount())
### 查看有多少个字段Field
for i in range(fea.GetFieldCount()):
    name_field = fea.GetFieldDefnRef(i).name
    print(name_field)
print()
print('Number of features:', layer.GetFeatureCount())
### 查看有多少个特征Feature
for i in range(layer.GetFeatureCount()):
    fea = layer.GetFeature(i)
    name_fea = fea.GetField('dt_name')
    print(name_fea)

ds = None


Number of layers: 1
Number of Features: 14
vector extent:  (214587.6929540675, 365546.4511921565, 2699511.4429286188, 2937162.819437231)
Number of Fields: 8
dt_adcode
dt_name
ct_adcode
ct_name
pr_adcode
pr_name
cn_adcode
cn_name

Number of features: 14
???
???
???
???
???
???
???
???
???
???????
???
?????????
?????????
???


#### 2. 添加/删除面积字段，并计算面积

In [12]:
ds = ogr.Open(path_kunming_utm, 1)  ## 0是只读，1是可写
layer = ds.GetLayer(0)
field_defn = ogr.FieldDefn('Area', ogr.OFTReal)   ## OFTReal：浮点型；OFTInteger: 整型
layer.CreateField(field_defn)       ## 创建新字段
# id_area = fea.GetFieldIndex('Area')
# layer.DeleteField(id_area)   ## 删除字段，参数只能是索引
# ds = None    ### 关闭数据源（重要）


0

In [13]:
for i in range(layer.GetFeatureCount()):
  fea = layer.GetFeature(i)
  fea_area = fea.geometry().Area()
  fea_area = fea_area/1000000       ### 单位：米
  print('Area is: ', fea_area)
  fea.SetField('Area', fea_area)   ## 对面积字段Field赋值
  layer.SetFeature(fea)  ## 更新特征
ds = None    ### 关闭数据源（重要）


Area is:  377.9959786954056
Area is:  864.2952967369265
Area is:  638.3336172947612
Area is:  897.3967316211479
Area is:  1866.1175875336044
Area is:  499.36396388767395
Area is:  1338.5644973111575
Area is:  994.6751193008348
Area is:  1914.0921754168446
Area is:  1681.3428560105956
Area is:  828.4756858595072
Area is:  4227.345074248606
Area is:  3598.3429000372385
Area is:  1301.0062596518349


#### 3. 获取面要素中心点，新建中心点图层

In [16]:
ds = ogr.Open(path_kunming, 0)  ## 0是只读，1是可写
layer = ds.GetLayer(0); 
sr = osr.SpatialReference()   # create spatial reference object
sr.ImportFromEPSG(4326)       # set it to EPSG:4326
driver = ogr.GetDriverByName('ESRI Shapefile')
ds_centroid = driver.CreateDataSource(path_kunming_centroid)
layer_centroid = ds_centroid.CreateLayer('centroid', geom_type=ogr.wkbPoint, srs=sr)  # create new layer in the shapefile 
field_defn = ogr.FieldDefn('Fid', ogr.OFTInteger)
layer_centroid.CreateField(field_defn)     ## 创建新字段
for i in range(layer.GetFeatureCount()):
  print(i+1)
  fea = layer.GetFeature(i)
  geo_centroid = fea.geometry().Centroid()  ## 获得中心点几何
  fea_centroid_defn = layer.GetLayerDefn()
  fea_centroid = ogr.Feature(fea_centroid_defn)   ## 创建空要素
  fea_centroid.SetGeometry(geo_centroid)     ## 设置要素几何
  fea_centroid.SetField('Fid', i+1)          ## 设置字段值
  layer_centroid.CreateFeature(fea_centroid)  ## 在图层中创建要素。
layer_centroid = ds_centroid = None       ## 关闭数据源和图层



1
2
3
4
5
6
7
8
9
10
11
12
13
14


ERROR 1: Invalid index : -1
ERROR 1: Invalid index : -1
ERROR 1: Invalid index : -1
ERROR 1: Invalid index : -1
ERROR 1: Invalid index : -1
ERROR 1: Invalid index : -1
ERROR 1: Invalid index : -1
ERROR 1: Invalid index : -1
ERROR 1: Invalid index : -1
ERROR 1: Invalid index : -1
ERROR 1: Invalid index : -1
ERROR 1: Invalid index : -1
ERROR 1: Invalid index : -1
ERROR 1: Invalid index : -1


#### 4. 新建几何及多要素图层
云大呈贡校园范围（(102.837,24.832), (102.842,24.836), (102.849,24.836), (102.857,24.835), (102.864,24.828),(102.855,24.826), (102.845,24.821),(102.837,24.832)）  
wkt(OGC well-known text)和wkb(OGC well-known binary)是OGC制定的空间数据的组织规范，wkt是以文本形式描述，wkb是以二进制形式描述。



In [64]:
driver = ogr.GetDriverByName('ESRI Shapefile')
ds_ynu = driver.CreateDataSource(path_ynu_vector)
sr = osr.SpatialReference()   ## create spatial reference object
sr.ImportFromEPSG(4326)       ## set it to EPSG:4326
layer_ynu = ds_ynu.CreateLayer('ynu_fromwkt', srs=sr, geom_type=ogr.wkbPolygon)
field_defn = ogr.FieldDefn('fid', ogr.OFTInteger)
layer_ynu.CreateField(field_defn)
fea_ynu = ogr.Feature(layer_ynu.GetLayerDefn())
fea_ynu.SetField('fid', 1)


In [26]:
# ### Create polygon by Adding Geometry()
# ring = ogr.Geometry(ogr.wkbLinearRing)
# ring.AddPoint(102.837,24.832)
# ring.AddPoint(102.842,24.836)
# ring.AddPoint(102.849,24.836)
# ring.AddPoint(102.857,24.835)
# ring.AddPoint(102.864,24.828)
# ring.AddPoint(102.855,24.826)
# ring.AddPoint(102.845,24.821)
# ring.AddPoint(102.837,24.832)
# # 创建多边形
# poly_ynu = ogr.Geometry(ogr.wkbPolygon)
# poly_ynu.AddGeometry(ring)
# print(poly_ynu.ExportToWkt())
# print(poly_ynu.ExportToWkb())


POLYGON ((102.837 24.832 0,102.842 24.836 0,102.849 24.836 0,102.857 24.835 0,102.864 24.828 0,102.855 24.826 0,102.845 24.821 0,102.837 24.832 0))
bytearray(b'\x01\x03\x00\x00\x80\x01\x00\x00\x00\x08\x00\x00\x00!\xb0rh\x91\xb5Y@\xa2E\xb6\xf3\xfd\xd48@\x00\x00\x00\x00\x00\x00\x00\x00\xd9\xce\xf7S\xe3\xb5Y@\xbct\x93\x18\x04\xd68@\x00\x00\x00\x00\x00\x00\x00\x00u\x93\x18\x04V\xb6Y@\xbct\x93\x18\x04\xd68@\x00\x00\x00\x00\x00\x00\x00\x00\x02+\x87\x16\xd9\xb6Y@\xf6(\\\x8f\xc2\xd58@\x00\x00\x00\x00\x00\x00\x00\x00\x9e\xef\xa7\xc6K\xb7Y@\x87\x16\xd9\xce\xf7\xd38@\x00\x00\x00\x00\x00\x00\x00\x00\x1f\x85\xebQ\xb8\xb6Y@\xfa~j\xbct\xd38@\x00\x00\x00\x00\x00\x00\x00\x00\xaeG\xe1z\x14\xb6Y@\x19\x04V\x0e-\xd28@\x00\x00\x00\x00\x00\x00\x00\x00!\xb0rh\x91\xb5Y@\xa2E\xb6\xf3\xfd\xd48@\x00\x00\x00\x00\x00\x00\x00\x00')


In [53]:
### Create polygon by define wkt
ynu_wkt = "POLYGON ((102.837 24.832,102.842 24.836,102.849 24.836,102.857 24.835,102.864 24.828,102.855 24.826,102.845 24.821,102.837 24.832))"
poly_ynu = ogr.CreateGeometryFromWkt(ynu_wkt)
poly_ynu


<osgeo.ogr.Geometry; proxy of <Swig Object of type 'OGRGeometryShadow *' at 0x7fc207be1930> >

In [54]:
fea_ynu.SetGeometry(poly_ynu)
layer_ynu.CreateFeature(fea_ynu)
ds_ynu = None


#### 课后作业
开发能够根据经度快速获得utm投影带的python函数。
